### importing required modules

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import os
import warnings
warnings.filterwarnings("ignore")
import shutil

### IP address masking
##### This is done because if you try scrape the site multiple site they will block you. So just change the country code and scrape again. 

In [2]:
l={} 
u=list() 
country_code="UK"
url="https://www.proxynova.com/proxy-server-list/country-"+country_code+"/"
respo = requests.get(url).text 
soup = bs(respo,'html.parser')
allproxy = soup.find_all("tr") 
for proxy in allproxy: 
    foo = proxy.find_all("td") 
    try:
        l["ip"]=foo[0].text.replace("\n","").replace("document.write(","").replace(")","").replace("\'","").replace(";","") 
    except:
        l["ip"]=None 
    try:
        l["port"]=foo[1].text.replace("\n","").replace(" ","") 
    except:
        l["port"]=None 
    try: 
        l["country"]=foo[5].text.replace("\n","").replace(" ","") 
    except:
        l["country"]=None
    if(l["port"] is not None):
        u.append(l) 
    l={}
    print(u)

## Scrapping the main site

In [3]:
url='https://seer.cancer.gov/statfacts/'
r= requests.get(url)
soup= bs(r.content)


In [4]:
##Creating the required directory
#get current directory
cwd=os.getcwd()
#create main directory for storing the data
cancer_directory='Cancer_Research'
path = os.path.join(cwd, cancer_directory)
isFile = os.path.exists(path)
if os.path.exists(path):
    shutil.rmtree(cwd+'\\'+cancer_directory)
    #os.remove(path)
os.mkdir(path)

In [5]:
##change current directory
os.chdir(path)
cwd=os.getcwd()

In [6]:
#create sub directory to store the data used
Age_directory='Age_Data'
path = os.path.join(cwd, Age_directory)
os.mkdir(path)
Race_directory='Ethnicity_Data'
path = os.path.join(cwd, Race_directory)
os.mkdir(path)
Stage_directory='Stage_Data'
path = os.path.join(cwd, Stage_directory)
os.mkdir(path)
Stage_directory='Trend_Data'
path = os.path.join(cwd, Stage_directory)
os.mkdir(path)

In [7]:
##Get the list of Cancer from the scraped data
All_Cancer_Data= soup.select('.col-md-4 a')

In [8]:
##Store the cancer list into an excel file and the link to the cancer data into a list
##Create complete link for different cancer type
Cancer_List=[]
Cancer_List_Link=[]
base_url='https://seer.cancer.gov/'
for i in range(0,len(All_Cancer_Data)):
    ld=[]
    cancer=All_Cancer_Data[i].get_text()
    link=base_url
    link+=All_Cancer_Data[i]['href']
    Cancer_List.append(cancer)
    ld.append(cancer)
    ld.append(link)
    Cancer_List_Link.append(ld)
df=pd.DataFrame(Cancer_List,columns=['Type Of Cancer'])
df.to_excel('CancerList.xlsx',index=False)

In [9]:
def observed_modeled_data(data, type):
    column_Name=[]
    head_data=data.find('thead')
    body_data=data.find('tbody')
    rows= head_data.find_all('tr')
    rows_head=rows[0].find_all('th')
    rowData=[]
    for i in range(0,len(rows_head)):
        th= rows_head[i].get_text()
        if i > 0:
            column_Name.append(th+'(Observed)')
            column_Name.append(th+'(Modeled)')
        else:
            column_Name.append(th)
    column_Name.append('Cancer Type')
    rows= body_data.find_all('tr')
    for i in range(0,len(rows)):
        rows_td=rows[i].find_all('td')
        total_td=[]
        for j in range(0,len(rows_td)):
            td=rows_td[j].get_text()
            total_td.append(td)
        total_td.append(type)
        rowData.append(total_td)
    df=pd.DataFrame(rowData)
    df.columns= column_Name
    return df

In [10]:
def five_year_data(data, type):
    column_Name=[]
    head_data=data.find('thead')
    body_data=data.find('tbody')
    rows= head_data.find_all('tr')
    rows_head=rows[0].find_all('th')
    rowData=[]
    for i in range(0,len(rows_head)):
        th= rows_head[i].get_text()
        column_Name.append(th)
    column_Name.append('Cancer Type')
    rows= body_data.find_all('tr')
    for i in range(0,len(rows)):
        row_th= rows[i].find('th').get_text();
        rows_td=rows[i].find_all('td')
        total_td=[]
        total_td.append(row_th)
        for j in range(0,len(rows_td)):
            td=rows_td[j].get_text()
            total_td.append(td)
        total_td.append(type)
        rowData.append(total_td)
    df=pd.DataFrame(rowData)
    df.columns= column_Name
    return df

In [11]:
def race_data(data, type, columnName, outcome):
    column_Name=columnName
    sex= data.find('h5').get_text()
    rows=data.find_all('tr')
    rowData=[]
    for i in range(0, len(rows)):
        total_td=[]
        th= rows[i].find('th').get_text()
        total_td.append(th)
        td= rows[i].find('td').get_text()
        total_td.append(td)
        total_td.append(sex)
        total_td.append(type)
        total_td.append(outcome)
        rowData.append(total_td)

    df=pd.DataFrame(rowData)
    df.columns= column_Name
    return df

In [12]:
def pie_chart_data(data, type, column_Name, outcome):
#     column_Name=[]
#     head_data=data.find('thead')
    body_data=data.find('tbody')
#     rows= head_data.find_all('tr')
#     rows_head=rows[0].find_all('th')
    rowData=[]
#     for i in range(0,len(rows_head)):
#         th= rows_head[i].get_text()
#         column_Name.append(th)
#     column_Name.append('Cancer Type')
    rows= body_data.find_all('tr')
    for i in range(0,len(rows)):
        rows_td=rows[i].find_all('td')
        total_td=[]
        for j in range(0,len(rows_td)):
            td=rows_td[j].get_text()
            total_td.append(td)
        total_td.append(type)
        total_td.append(outcome)
        rowData.append(total_td)
    df=pd.DataFrame(rowData)
    df.columns= column_Name
    return df

In [13]:
### Data type 1 which has a total of 8 scrapeable table
def getData_Type1(soup1, type):
    table_data=[]
    for table in soup1.select('div.scrapeTable'):
        table_data.append(table)
    for i in range(0,len(table_data)):
        if i==0:
            #Observed and modeled data over the years
            result0= observed_modeled_data(table_data[0], type)
        elif i==1:
            #Five year new cases and deaths data
            result1= five_year_data(table_data[1], type)
        elif i==2:
            #New cases data based on the ethinicity for Males
            result2= race_data(table_data[2],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'New Cases')
        elif i==3:
            #New cases data based on the ethinicity for Females
            result3= race_data(table_data[3],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'New Cases') 
        elif i==4:
            #New cases data based on the age of patients
            result4= pie_chart_data(table_data[4], type,['Age Range','Percent of New Cases/ Deaths', 'Cancer Type', 'Outcome'], 'New Cases')
        elif i==5:
            #Deaths data based on the ethinicity for Males
            result5= race_data(table_data[5],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'Deaths')
        elif i==6:
            #Deaths data based on the ethinicity for Females
            result6= race_data(table_data[6],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'Deaths')
        elif i==7:
            #Deaths data based on the age of the patients
            result7= pie_chart_data(table_data[7], type,['Age Range','Percent of New Cases/ Deaths', 'Cancer Type', 'Outcome'], 'Deaths')
    
    ##Creating dataframe
    result0df= pd.DataFrame(result0)
    result1df= pd.DataFrame(result1)
    result2df= pd.DataFrame(result2)
    result3df= pd.DataFrame(result3)
    result4df= pd.DataFrame(result4)
    result5df= pd.DataFrame(result5)
    result6df= pd.DataFrame(result6)
    result7df= pd.DataFrame(result7)
    
     
    ##Concating same type of data
    Age_Data_List= [result4, result7]
    Ethnicity_Data_List= [result2, result3, result5, result6]
    Trend_Data_List= [result0]
    Stage_Data_List= [result1]
    
    ##Creating dataframe
    Age_Data= pd.concat(Age_Data_List)
    Ethnicity_Data= pd.concat(Ethnicity_Data_List)
    Trend_Data= pd.concat(Trend_Data_List)
    Stage_Data= pd.concat(Stage_Data_List)
    
    #Creating file name
    age_file= './Age_Data/'+type+'_age_data.xlsx'
    ethnic_file= './Ethnicity_Data/'+type+'_ethnicity_data.xlsx'
    trend_file= './Trend_Data/'+type+'_trend_data.xlsx'
    Stage_file= './Stage_Data/'+type+'_stage_data.xlsx'
    
    #Save to excel
    Age_Data.to_excel(age_file, index=False)
    Ethnicity_Data.to_excel(ethnic_file, index=False)
    Trend_Data.to_excel(trend_file, index=False)
    Stage_Data.to_excel(Stage_file, index=False)
            

In [14]:
### Data type 2 which has a total of 7 scrapeable table
def getData_Type2(soup1, type):
    table_data=[]
    for table in soup1.select('div.scrapeTable'):
        table_data.append(table)
    for i in range(0,len(table_data)):
        if i==0:
            #Observed and modeled data over the years
            result0= observed_modeled_data(table_data[0], type)
        elif i==1:
            #New cases data based on the ethinicity for Males
            result1= race_data(table_data[1],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'New Cases')
        elif i==2:
            #New cases data based on the ethinicity for Females
            result2= race_data(table_data[2],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'New Cases') 
        elif i==3:
            #New cases data based on the age of patients
            result3= pie_chart_data(table_data[3], type,['Age Range','Percent of New Cases/ Deaths', 'Cancer Type', 'Outcome'], 'New Cases')
        elif i==4:
            #Deaths data based on the ethinicity for Males
            result4= race_data(table_data[4],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'Deaths')
        elif i==5:
            #Deaths data based on the ethinicity for Females
            result5= race_data(table_data[5],type,['Ethnicity','per 100000','SEX', 'Cancer Type', 'New Cases / Deaths'], 'Deaths')
        elif i==6:
            #Deaths data based on the age of the patients
            result6= pie_chart_data(table_data[6], type,['Age Range','Percent of New Cases/ Deaths', 'Cancer Type', 'Outcome'], 'Deaths')
    
    ##Concating same type of data
    Age_Data_List= [result3, result6]
    Ethnicity_Data_List= [result1, result2, result4, result5]
    Trend_Data_List= [result0]
    
    ##Creating dataframe
    Age_Data= pd.concat(Age_Data_List)
    Ethnicity_Data= pd.concat(Ethnicity_Data_List)
    Trend_Data= pd.concat(Trend_Data_List)
    
    #Creating file name
    age_file= './Age_Data/'+type+'_age_data.xlsx'
    ethnic_file= './Ethnicity_Data/'+type+'_ethnicity_data.xlsx'
    trend_file= './Trend_Data/'+type+'_trend_data.xlsx'
    
    #Save to excel
    Age_Data.to_excel(age_file, index=False)
    Ethnicity_Data.to_excel(ethnic_file, index=False)
    Trend_Data.to_excel(trend_file, index=False)
            

In [15]:
##### Going through the cancer list and scraping the data for each cancer types
for i in range(0,len(Cancer_List_Link)):
    title= Cancer_List_Link[i][0]
    link=Cancer_List_Link[i][1]
    r1= requests.get(link)
    soup1= bs(r1.content)
    if title =='Anus' or title == 'Bladder' or title == 'Breast (Female)' or title == 'Cervix Uteri' or title == 'Childhood Brain and Other Nervous System (Ages 0–19)' or title == 'Colon and Rectum' or title == 'Esophagus' or title == 'Kidney and Renal Pelvis' or title == 'Larynx' or title == 'Liver and Intrahepatic Bile Duct' or title == 'Lung and Bronchus' or title == 'Melanoma of the Skin' or title == 'Oral Cavity and Pharynx' or title == 'Lip' or title == 'Tongue' or title == 'Ovary'or title == 'Pancreas' or title == 'Prostate' or title == 'Small Intestine' or title == 'Soft Tissue including Heart' or title == 'Stomach' or title == 'Thyroid' or title == 'Uterus'or title == 'Vulva':
        getData_Type1(soup1, title)
    elif title == 'Bone and Joint' or title == 'Childhood Leukemia (Ages 0–19)' or title == 'Leukemia' or title == 'Testis':
        getData_Type2(soup1, title)
    else:
        continue

In [16]:
## Concating all the same types of data of various cancer types into one excel file

Ethnic_Data_Complete=pd.DataFrame()
Age_Data_Complete=pd.DataFrame()
Trend_Data_Complete=pd.DataFrame()
stage_data_Complete=pd.DataFrame()

files=[file for file in os.listdir('./Age_Data')]
for file in files:
    df=pd.read_excel('./Age_Data/'+file,index_col=None)
    Age_Data_Complete=pd.concat([Age_Data_Complete,df])
    
files=[file for file in os.listdir('./Ethnicity_Data')]
for file in files:
    df=pd.read_excel('./Ethnicity_Data/'+file,index_col=None)
    Ethnic_Data_Complete=pd.concat([Ethnic_Data_Complete,df])
    
files=[file for file in os.listdir('./Trend_Data')]
for file in files:
    df=pd.read_excel('./Trend_Data/'+file,index_col=None)
    Trend_Data_Complete=pd.concat([Trend_Data_Complete,df])
    
files=[file for file in os.listdir('./Stage_Data')]
for file in files:
    df=pd.read_excel('./Stage_Data/'+file,index_col=None)
    stage_data_Complete=pd.concat([stage_data_Complete,df])

##Save the data into excl files
Trend_Data_Complete.to_excel('Trend_Data_Complete.xlsx',index=False)
Age_Data_Complete.to_excel('Age_Data_Complete.xlsx',index=False)
Ethnic_Data_Complete.to_excel('Ethnic_Data_Complete.xlsx',index=False)
stage_data_Complete.to_excel('stage_data_Complete.xlsx',index=False)